In [554]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore')

In [591]:
positions = pd.read_csv("portfolio_positions_exploded.csv")
costs = pd.read_csv("portfolio_costs_exploded.csv")

Construção do PNL de Posições

DTD

In [ ]:
positions['overview_date'] = pd.to_datetime(positions['overview_date'])
positions = positions.sort_values(['portfolio_id','instrument_id','portfolio_origem','overview_date'])

# Pego exposição e nav do dia anterior pra calcular o pnl diário
positions['exposure_value_ontem'] = (
    positions.groupby(['portfolio_id','instrument_id','portfolio_origem'])['exposure_value']
    .shift(1)
)
positions['portfolio_nav_ontem'] = (
    positions.groupby(['portfolio_id'])['portfolio_nav']
    .shift(1)
)

# Calculo do pnl diário do ativo e da carteira
positions['dtd_ativo_pct'] = positions['dtd_ativo_fin'] / positions['exposure_value_ontem']
positions['dtd_carteira_pct'] = positions['dtd_ativo_fin'] / positions['portfolio_nav_ontem']

MTD

In [ ]:
positions['Year'] = positions['overview_date'].dt.year
positions['Month'] = positions['overview_date'].dt.month

# Calcula ano/mês do mês anterior
positions['Month_after'] = positions['Month'] + 1
positions['Year_after'] = positions['Year']
positions.loc[positions['Month_after'] == 13, 'Month_after'] = 1
positions.loc[positions['Month'] == 12, 'Year_after'] += 1

# Tabela com último exposure_value do mês para cada grupo
last_exposure_month = (
    positions
    .groupby(['portfolio_id','instrument_id','portfolio_origem','Year','Month'])
    .apply(lambda g: g.loc[g['overview_date'] == g['overview_date'].max()])
    .reset_index(drop=True)
)[['portfolio_id','instrument_id','portfolio_origem','Year_after','Month_after','exposure_value']]

last_exposure_month = last_exposure_month.rename(
    columns={'Year_after':'Year','Month_after':'Month','exposure_value':'exposure_value_prev'}
)

# Faz merge para trazer o exposure_value do mes anterior para cada linha
positions = positions.merge(
    last_exposure_month,
    on=['portfolio_id','instrument_id','portfolio_origem','Year','Month'],
    how='left'
)

# Tabela com último portfolio_nav do mês para cada grupo (faço o groupby de novo pra não incluir o instrument_id aqui)
last_nav_month = (
    positions
    .groupby(['portfolio_id','portfolio_origem','Year','Month'])
    .apply(lambda g: g.loc[g['overview_date'] == g['overview_date'].max()])
    .reset_index(drop=True)
)[['portfolio_id','portfolio_origem','Year_after','Month_after','portfolio_nav']]

last_nav_month = last_nav_month.rename(
    columns={'Year_after':'Year','Month_after':'Month', 'portfolio_nav':'portfolio_nav_prev'}
)

last_nav_month = last_nav_month.drop_duplicates()

# Faz merge para trazer o nav do mes anterior para cada linha
positions = positions.merge(
    last_nav_month,
    on=['portfolio_id','portfolio_origem','Year','Month'],
    how='left'
)

positions = positions.sort_values(['portfolio_id','instrument_id','portfolio_origem','overview_date'])


# Calcula o PnL MTD financeiro pra todos os dias:
positions['mtd_ativo_fin'] = positions.groupby(
    ['portfolio_id', 'instrument_id', 'portfolio_origem', 'Year', 'Month']
)['dtd_ativo_fin'].cumsum()

positions['mtd_ativo_pct'] = positions['mtd_ativo_fin'] / positions['exposure_value_prev']

positions['mtd_carteira_pct'] = positions['mtd_ativo_fin'] / positions['portfolio_nav_prev']

Expansão do PNL MTD para ativos que foram zerados ao longo do mês

In [597]:
# PnL MTD
group_cols = ['portfolio_id','instrument_id','portfolio_origem','Year','Month']
all_dates = positions.groupby(group_cols)['overview_date'].agg(['min', 'max']).reset_index()

# Gera linhas para todos os dias do mês para cada ativo
expanded = []
for _, row in all_dates.iterrows():
    year = row['Year']
    month = row['Month']
    filtro = (positions['overview_date'].dt.year == year) & (positions['overview_date'].dt.month == month)

    # pega o último dia do mês de forma automática
    dates = list(positions['overview_date'][filtro].drop_duplicates())
    for date in dates:
        expanded.append({**row, 'overview_date': date})

calendar = pd.DataFrame(expanded)

# Passo 2: Merge calendar com positions
positions_full = pd.merge(calendar, positions, on=group_cols + ['overview_date'], how='left')

# Passo 3: Forward fill do PnL e das outras colunas por ativo/mês
positions_full = positions_full.sort_values(group_cols + ['overview_date'])
positions_full['mtd_carteira_pct'] = positions_full.groupby(group_cols)['mtd_carteira_pct'].ffill()

In [600]:
# Tratamento do DF para exportação
positions_full_filtrado = positions_full[[
    'portfolio_id','portfolio_origem','overview_date','instrument_id',
    'dtd_ativo_fin','dtd_ativo_pct','dtd_carteira_pct', 'mtd_ativo_fin',
    'mtd_ativo_pct','mtd_carteira_pct']]
positions_full_filtrado.replace([np.inf, -np.inf], 0, inplace=True)

positions_full_filtrado.to_csv("positions_pnl_history.csv", index=False)


Construção do PNL de Custos

In [560]:
costs['overview_date'] = pd.to_datetime(costs['overview_date'])

costs['Year'] = costs['overview_date'].dt.year
costs['Month'] = costs['overview_date'].dt.month

costs = costs.sort_values(['origin_portfolio_id', 'root_portfolio', 'overview_date'])

# Calcula ano/mês do mês anterior
costs['Month_after'] = costs['Month'] + 1
costs['Year_after'] = costs['Year']
costs.loc[costs['Month_after'] == 13, 'Month_after'] = 1
costs.loc[costs['Month'] == 12, 'Year_after'] += 1

# Calcula o PnL MTD pra qualquer dia:
costs['mtd_custos_fin'] = costs.groupby(
    ['category_name','origin_portfolio_id','root_portfolio','Year','Month']
)['dtd_custos_fin'].cumsum()

costs['dtd_custos_pct'] = costs['dtd_custos_fin'] / costs['portfolio_nav_ontem']
costs['mtd_custos_pct'] = costs['mtd_custos_fin'] / costs['portfolio_nav_ontem']

In [561]:
# PnL MTD
group_cols = ['category_name','origin_portfolio_id','root_portfolio','Year','Month']
all_dates = costs.groupby(group_cols)['overview_date'].agg(['min', 'max']).reset_index()

# Gera linhas para todos os dias do mês para cada ativo
expanded = []
for _, row in all_dates.iterrows():
    year = row['Year']
    month = row['Month']
    filtro = (costs['overview_date'].dt.year == year) & (costs['overview_date'].dt.month == month)

    # pega o último dia do mês de forma automática
    dates = list(costs['overview_date'][filtro].drop_duplicates())
    for date in dates:
        expanded.append({**row, 'overview_date': date})

calendar = pd.DataFrame(expanded)

# Passo 2: Merge calendar com costs
costs_full = pd.merge(calendar, costs, on=group_cols + ['overview_date'], how='left')

# Passo 3: Forward fill do PnL e das outras colunas por ativo/mês
costs_full = costs_full.sort_values(group_cols + ['overview_date'])
costs_full['mtd_custos_pct'] = costs_full.groupby(group_cols)['mtd_custos_pct'].ffill()

In [562]:
costs_full.to_clipboard()

In [563]:
# Tratamento do DF para exportação
costs_full_filtrado = costs_full[[
    'overview_date','category_name','origin_portfolio_id','root_portfolio',
    'dtd_custos_fin', 'portfolio_id',
    'mtd_custos_fin','dtd_custos_pct','mtd_custos_pct']]

costs_full_filtrado.replace([np.inf, -np.inf], 0, inplace=True)
costs_full_filtrado.to_csv("costs_pnl_history.csv", index=False)


In [564]:
positions

,overview_date,portfolio_id,instrument_id,book_name,instrument_type,quantity,price,asset_value,exposure_value,dtd_ativo_fin,...,dtd_carteira_pct,Year,Month,Month_after,Year_after,exposure_value_prev,portfolio_nav_prev,mtd_ativo_fin,mtd_ativo_pct,mtd_carteira_pct
0,2025-05-30,1157,1,Caixa,7,1.009100e+02,1.000000,100.91,1.009100e+02,0.00,...,NaN,2025,5,6,2025,NaN,NaN,0.00,NaN,NaN
1,2025-06-02,1157,1,Caixa,7,1.009100e+02,1.000000,100.91,1.009100e+02,0.00,...,0.000000,2025,6,7,2025,1.009100e+02,7.503970e+07,0.00,0.000000,0.000000
2,2025-06-03,1157,1,Caixa,7,1.009100e+02,1.000000,100.91,1.009100e+02,0.00,...,0.000000,2025,6,7,2025,1.009100e+02,7.503970e+07,0.00,0.000000,0.000000
3,2025-06-04,1157,1,Caixa,7,1.009100e+02,1.000000,100.91,1.009100e+02,0.00,...,0.000000,2025,6,7,2025,1.009100e+02,7.503970e+07,0.00,0.000000,0.000000
4,2025-06-05,1157,1,Caixa,7,1.008200e+02,1.000000,100.82,1.008200e+02,0.00,...,0.000000,2025,6,7,2025,1.009100e+02,7.503970e+07,0.00,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,2025-06-20,1886,1265,Risco >> Multimercado Terceiros,3,2.585508e+07,1.108759,28667052.05,2.866705e+07,-91834.15,...,-0.000619,2025,6,7,2025,2.834751e+07,1.467322e+08,319541.92,0.011272,0.002178
8042,2025-06-23,1886,1265,Risco >> Multimercado Terceiros,3,2.585508e+07,1.108860,28669675.82,2.866968e+07,2623.77,...,0.000018,2025,6,7,2025,2.834751e+07,1.467322e+08,322165.69,0.011365,0.002196
8043,2025-06-24,1886,1265,Risco >> Multimercado Terceiros,3,2.585508e+07,1.117208,28885508.10,2.888551e+07,215832.28,...,0.001448,2025,6,7,2025,2.834751e+07,1.467322e+08,537997.97,0.018979,0.003667
8044,2025-06-25,1886,1265,Risco >> Multimercado Terceiros,3,2.585508e+07,1.115825,28849746.39,2.884975e+07,-35761.72,...,-0.000238,2025,6,7,2025,2.834751e+07,1.467322e+08,502236.25,0.017717,0.003423
